# Running Different Permutations of Models
- example below is using the car data set and the cros_val_score to test different Decision Tree models for different X-variables
- first section is doing it manually, second section I've automated with a for loop
- I've also automated looking at different scoring metrics

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import tree
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import (KFold, StratifiedKFold, GroupKFold)

#Codes for logistic
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

In [9]:
df = pd.read_csv("ExData\car_data.csv", index_col=['User ID'])
df = pd.concat([df, pd.get_dummies(df['Gender'])], axis=1)
df['AnnualSalaryStd'] = StandardScaler().fit_transform(df[['AnnualSalary']])

https://scikit-learn.org/stable/modules/model_evaluation.html

# Permutate Models

In [33]:
import itertools
allcols = ['Age', 'AnnualSalaryStd', 'Female']
allXs = []
for num in range(1,len(allcols)+1):
    combos = [list(x) for x in itertools.combinations(allcols, num)]
    allXs.extend(combos)
len(allXs), allXs

(7,
 [['Age'],
  ['AnnualSalaryStd'],
  ['Female'],
  ['Age', 'AnnualSalaryStd'],
  ['Age', 'Female'],
  ['AnnualSalaryStd', 'Female'],
  ['Age', 'AnnualSalaryStd', 'Female']])

In [34]:
clf1 = tree.DecisionTreeClassifier(random_state=42)
clf2 = LogisticRegression(random_state=42)
typesModels = {'Decision':clf1, 'LogReg':clf2}
y = df['Purchased']
metrics = ['accuracy','recall','precision','f1','roc_auc']

In [35]:
X_train, X_test, y_train, y_test = train_test_split(df[allcols], y, test_size=0.2, random_state=0)
        #800 rows of training, 200 of testing

In [55]:
results = {}
for type, clf in typesModels.items(): #type ='Decision' or 'LogReg', clf = the model
    for metric in metrics:
        models = []
        for cols in allXs:   
            #print(type, cols)
            scores = cross_val_score(clf, X_train[cols], y_train, cv=5, scoring=metric)
                            #only using the 800 training rows, but does more splits 
                            #run 5 tests --> each test might be doing 100 test and 700 train
            models.append({'Model':type,'Scores':scores,'Mean':scores.mean(),
                           'STD':scores.std(),'Min':scores.min(),'Cols':cols})
        resultsTable = pd.DataFrame(models)
        results[(metric,type)] = resultsTable

c:\users\v806013\ds\tools\python3.8\latest\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\v806013\ds\tools\python3.8\latest\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\v806013\ds\tools\python3.8\latest\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\v806013\ds\tools\python3.8\latest\lib\site-packages\skl

In [43]:
allModelsDF = pd.concat(results)
allModelsDF.to_excel('Output/Models.xlsx')
allModelsDF.head(50)

Model  \
accuracy  Decision 0  Decision   
                   1  Decision   
                   2  Decision   
                   3  Decision   
                   4  Decision   
                   5  Decision   
                   6  Decision   
recall    Decision 0  Decision   
                   1  Decision   
                   2  Decision   
                   3  Decision   
                   4  Decision   
                   5  Decision   
                   6  Decision   
precision Decision 0  Decision   
                   1  Decision   
                   2  Decision   
                   3  Decision   
                   4  Decision   
                   5  Decision   
                   6  Decision   
f1        Decision 0  Decision   
                   1  Decision   
                   2  Decision   
                   3  Decision   
                   4  Decision   
                   5  Decision   
                   6  Decision   
roc_auc   Decision 0  Decision   
                   1  Decision   
                   2  Decision   
                   3  Decision   
                   4  Decision   
                   5  Decision   
                   6  Decision   
accuracy  LogReg   0    LogReg   
                   1    LogReg   
                   2    LogReg   
                   3    LogReg   
                   4    LogReg   
                   5    LogReg   
                   6    LogReg   
recall    LogReg   0    LogReg   
                   1    LogReg   
                   2    LogReg   
                   3    LogReg   
                   4    LogReg   
                   5    LogReg   
                   6    LogReg   
precision LogReg   0    LogReg   

                                                                 Scores  \
accuracy  Decision 0              [0.8, 0.85, 0.76875, 0.83125, 0.8375]   
                   1         [0.70625, 0.71875, 0.70625, 0.725, 0.7125]   
                   2              [0.6, 0.6, 0.59375, 0.59375, 0.59375]   
                   3             [0.8, 0.90625, 0.85625, 0.85, 0.84375]   
                   4          [0.78125, 0.825, 0.7625, 0.80625, 0.8375]   
                   5          [0.6875, 0.725, 0.71875, 0.7375, 0.69375]   
                   6                [0.8, 0.91875, 0.88125, 0.85, 0.85]   
recall    Decision 0  [0.671875, 0.71875, 0.5846153846153846, 0.7230...   
                   1  [0.546875, 0.53125, 0.5076923076923077, 0.5230...   
                   2                          [0.0, 0.0, 0.0, 0.0, 0.0]   
                   3  [0.75, 0.859375, 0.7846153846153846, 0.8, 0.81...   
                   4  [0.65625, 0.640625, 0.5692307692307692, 0.7230...   
                   5  [0.53125, 0.59375, 0.5384615384615384, 0.61538...   
                   6  [0.703125, 0.859375, 0.8153846153846154, 0.846...   
precision Decision 0  [0.7962962962962963, 0.8846153846153846, 0.791...   
                   1  [0.660377358490566, 0.6938775510204082, 0.6875...   
                   2                          [0.0, 0.0, 0.0, 0.0, 0.0]   
                   3  [0.75, 0.9016393442622951, 0.85, 0.82539682539...   
                   4  [0.7636363636363637, 0.8913043478260869, 0.787...   
                   5  [0.6296296296296297, 0.6785714285714286, 0.7, ...   
                   6  [0.7758620689655172, 0.9322033898305084, 0.883...   
f1        Decision 0  [0.728813559322034, 0.7931034482758621, 0.6725...   
                   1  [0.5982905982905983, 0.6017699115044248, 0.584...   
                   2                          [0.0, 0.0, 0.0, 0.0, 0.0]   
                   3    [0.75, 0.88, 0.816, 0.8125, 0.8091603053435115]   
                   4  [0.7058823529411765, 0.7454545454545454, 0.660...   
                   5  [0.5762711864406779, 0.6333333333333334, 0.608...   
                   6  [0.7377049180327868, 0.8943089430894309, 0.848...   
roc_auc   Decision 0  [0.8541666666666666, 0.8712565104166667, 0.761...   
                   1  [0.724609375, 0.74

In [57]:
#Final Model
finalX = X_train[['Age', 'AnnualSalaryStd']]
clf1.fit(finalX, y_train)

DecisionTreeClassifier(random_state=42)

In [58]:
clf1.score(finalX, y_train)

0.9925

In [59]:
clf1.score(X_test[['Age', 'AnnualSalaryStd']],y_test)

0.905